In [1]:
import sys
sys.path.append("../")
import genjax
import numpy as np
import bayes3d as b
import jax.numpy as jnp
import bayes3d.genjax
import jax
from utils import *
from viz import *
from models import *
from renderer_setup import *

console = genjax.pretty()

In [4]:
gt_path = "../ground_truths/genjax_generated/scene_1.pkl"
metadata = load_metadata(gt_path)
gt_images = metadata["rendered"]
RENDERER_ARGS = metadata["RENDERER_ARGS"]
model = eval("model_v{}".format(metadata["model_version"]))
setup_renderer_and_meshes = eval("setup_renderer_and_meshes_v{}".format(metadata["renderer_setup_version"]))
model_importance_jit = eval("model_v{}_importance_jit".format(metadata["model_version"]))
model_simulate_jit = eval("model_v{}_simulate_jit".format(metadata["model_version"]))

setup_renderer_and_meshes(**RENDERER_ARGS)

[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


Increasing frame buffer size to (width, height, depth) = (64, 64, 1024)
Centering mesh with translation [ 8.9965761e-07  2.3238501e-02 -3.4500263e-06]
Centering mesh with translation [0.        0.0063132 0.       ]
Centering mesh with translation [ 5.9977174e-07  2.3238501e-02 -3.4500263e-06]
Centering mesh with translation [0.         0.05167415 0.        ]


In [20]:
# renderer initialized outside this
def inference_approach_A(model_fs, gt, metadata, num_particles):
    (model, model_importance_jit, model_simulate_jit) = model_fs
    chm = genjax.choice_map(metadata["CHOICE_MAP_ARGS"])
    # force new constaints values to take over
    chm = chm.unsafe_merge(genjax.choice_map(
        {"depths" : genjax.vector_choice_map(genjax.choice_map({
                "depths": gt
        }))}))
    
    


In [ ]:
def intphys_experiment(scene, inference_approach):
    pass